# Find proteins within the 7q gain event (cis) that are commonly different between samples with and without the event

## Setup

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
CHROMOSOME = "7"
ARM = "q"
TRANS_OR_CIS = "cis"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_ttest.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,brca_Database_ID,luad_Database_ID,ccrcc_Database_ID,lscc_Database_ID,ovarian_Database_ID,brca_pvalue,ccrcc_pvalue,colon_pvalue,endometrial_pvalue,gbm_pvalue,...,luad_pvalue,ovarian_pvalue,brca_diff,ccrcc_diff,colon_diff,endometrial_diff,gbm_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,,,,,,
AASS,NP_005754.2,NP_005754.2,NP_005754.2,NP_005754.2,NP_005754,0.892013,0.176283,0.332356,0.425675,0.251765,...,0.836283,0.290163,0.138533,0.193225,-0.149578,0.402394,0.249802,-0.873094,0.259281,0.285441
ABCB8,NP_001269220.1|NP_009119.2|NP_001269222.1|NP_0...,NP_001269220.1|NP_009119.2|NP_001269222.1|NP_0...,NP_001269222.1,NP_001269220.1|NP_009119.2|NP_001269222.1|NP_0...,NP_001269220,0.901831,0.076895,0.049230,0.051964,0.014576,...,0.254957,0.972555,0.100520,0.226167,0.226265,0.638488,0.342897,-0.274635,0.550333,0.015632
ABCF2,NP_005683.2|NP_009120.1,NP_005683.2|NP_009120.1,NP_005683.2,NP_005683.2|NP_009120.1,NP_005683,0.038583,0.006878,0.176363,0.775370,0.092395,...,0.026839,0.115655,0.725125,0.202061,0.129441,0.048771,0.238230,0.408913,0.634412,0.223571
ABHD11,NP_683710.1|NP_683711.1|NP_001138836.1|NP_0013...,NP_683710.1|NP_683711.1|NP_001138836.1|NP_0013...,NP_683710.1,NP_683710.1|NP_683711.1|NP_001138836.1|NP_0013...,NP_683710,0.721898,0.235208,0.222029,0.494209,0.522541,...,0.024617,0.586972,0.342268,0.160789,0.226416,0.309519,0.144931,0.292412,1.515471,0.218219
AGAP3,NP_114152.3|NP_001268229.1|NP_001036000.1|NP_0...,NP_114152.3|NP_001337031.1|NP_001268229.1|NP_0...,NP_001337031.1,NP_114152.3|NP_001337031.1|NP_001268229.1|NP_0...,NP_114152,0.856483,0.035356,0.810125,0.659057,0.346898,...,0.015437,0.277909,-0.141465,0.158475,-0.050370,0.098123,0.176082,0.318962,0.717756,0.182623
AGAP3,NP_114152.3|NP_001268229.1|NP_001036000.1|NP_0...,NP_114152.3|NP_001337031.1|NP_001268229.1|NP_0...,NP_001337031.1,NP_114152.3|NP_001337031.1|NP_001268229.1|NP_0...,NP_001295234,0.856483,0.035356,0.810125,0.659057,0.346898,...,0.015437,0.309097,-0.141465,0.158475,-0.050370,0.098123,0.176082,0.318962,0.717756,0.203818
AGFG2,NP_006067.3,NP_006067.3,NP_006067.3,NP_006067.3,NP_006067,0.982037,0.286067,0.223166,0.207530,0.041799,...,0.092956,0.088552,0.010062,0.106189,0.191423,0.420630,0.361124,-0.097457,0.710233,0.315298
AGK,NP_060708.1,NP_060708.1,NP_060708.1,NP_060708.1,NP_060708,0.097680,0.031408,0.000884,0.068854,0.010768,...,0.195861,0.088552,0.699184,0.279574,0.305651,0.452911,0.351636,0.205472,0.512687,0.279691
AHCYL2,NP_001124192.1|NP_056143.1,NP_001124192.1|NP_056143.1|NP_001124194.2,NaN,NP_001124192.1|NP_056143.1|NP_001124194.2,NP_001124192,0.916489,NaN,0.855175,0.024134,0.056098,...,0.186990,0.404875,0.075767,NaN,-0.039469,0.575819,0.464151,0.110541,0.699702,0.140517


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,AASS,NP_005754.2,0.892013,0.138533
1,brca,ABCB8,NP_001269220.1|NP_009119.2|NP_001269222.1|NP_0...,0.901831,0.100520
2,brca,ABCF2,NP_005683.2|NP_009120.1,0.038583,0.725125
3,brca,ABHD11,NP_683710.1|NP_683711.1|NP_001138836.1|NP_0013...,0.721898,0.342268
4,brca,AGFG2,NP_006067.3,0.982037,0.010062
5,brca,AGK,NP_060708.1,0.097680,0.699184
6,brca,AHCYL2,NP_001124192.1|NP_056143.1,0.916489,0.075767
7,brca,AKAP9,NP_671714.1,0.148652,0.551719
8,brca,AKR1B1,NP_001619.1,0.752628,-0.246709
9,brca,AKR1B10,NP_064695.3,0.927273,-0.155067


## Select the proteins with a significant change

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)

In [7]:
prots.groupby("cancer_type").count()[["protein"]]

,protein
cancer_type,
brca,6
ccrcc,39
colon,24
endometrial,16
gbm,15
lscc,20
luad,48
ovarian,24


## Find how many cancers each protein was different in

In [8]:
def make_simple_change(change_val):
    if change_val == 0:
        return 0
    if change_val > 0:
        return 1
    if change_val < 0:
        return -1

prots = prots.assign(
    simplified_change=prots["change"].apply(make_simple_change)
)

In [9]:
prots

,cancer_type,protein,Database_ID,adj_p,change,simplified_change
0,brca,ABCF2,NP_005683.2|NP_009120.1,0.038583,0.725125,1
1,brca,ARF5,NP_001653.1,0.024899,0.923086,1
2,brca,CEP41,NP_061188.1|NP_001244087.1|NP_001244088.1,0.022394,1.077799,1
3,brca,PEG10,NP_001165908.1|NP_001171890.1|NP_055883.2,0.038583,2.429459,1
4,brca,SBDS,NP_057122.2,0.022394,0.809829,1
5,brca,TRIP6,NP_003293.2,0.038583,1.020360,1
6,ccrcc,ABCF2,NP_005683.2,0.006878,0.202061,1
7,ccrcc,AGK,NP_060708.1,0.031408,0.279574,1
8,ccrcc,ALKBH4,NP_060091.1,0.041765,0.139707,1
9,ccrcc,ARF5,NP_001653.1,0.000508,0.303766,1


In [10]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist()),
    "mean_simp_change": ("simplified_change", np.mean)
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,mean_simp_change,num_cancers
protein,,,
ARF5,"[brca, ccrcc, colon, lscc, luad]",1,5
CDK5,"[ccrcc, colon, endometrial, lscc, luad]",1,5
TNPO3,"[ccrcc, colon, lscc, luad, ovarian]",1,5
WASL,"[ccrcc, endometrial, gbm, lscc, luad]",1,5
YWHAG,"[colon, endometrial, gbm, lscc, luad]",1,5
TRIP6,"[brca, lscc, luad, ovarian]",1,4
FIS1,"[ccrcc, colon, endometrial, luad]",1,4
SSBP1,"[ccrcc, colon, gbm, luad]",1,4
ARHGEF5,"[ccrcc, colon, lscc, luad]",1,4


In [11]:
prots_summary[prots_summary["cancers"].apply(lambda x: "gbm" in x)]

,cancers,mean_simp_change,num_cancers
protein,,,
WASL,"[ccrcc, endometrial, gbm, lscc, luad]",1,5
YWHAG,"[colon, endometrial, gbm, lscc, luad]",1,5
SSBP1,"[ccrcc, colon, gbm, luad]",1,4
UBE3C,"[ccrcc, gbm, lscc, luad]",1,4
AGK,"[ccrcc, colon, gbm]",1,3
PDAP1,"[ccrcc, colon, gbm]",1,3
TECPR1,"[ccrcc, gbm, luad]",1,3
UBE2H,"[ccrcc, gbm, luad]",1,3
CNOT4,"[gbm, lscc, ovarian]",1,3


## Save results

In [12]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv"
prots_summary.to_csv(output_file, sep="\t")